In [170]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 


In [171]:
## include results 

results = pd.read_csv("random_results.xlsx")

results_noise0 = results[["group", "size", "size_class", "vars", "correct"]]
results_noise0['correct'] = results_noise0['correct'].apply(lambda x: 100 * x)

results_noise_0001 = results[["group", "size", "size_class", "correct.1"]]
results_noise_0001['correct.2'] = results_noise_0001['correct.1'].apply(lambda x: 100 * x)

results_noise_001 = results[["group", "size", "size_class", "correct.2"]]
results_noise_001['correct.2'] = results_noise_001['correct.2'].apply(lambda x: 100 * x)


/tmp/ipykernel_18344/884508441.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_noise0['correct'] = results_noise0['correct'].apply(lambda x: 100 * x)
/tmp/ipykernel_18344/884508441.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_noise_0001['correct.2'] = results_noise_0001['correct.1'].apply(lambda x: 100 * x)
/tmp/ipykernel_18344/884508441.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [172]:
############################# real instances (groundtruth): AI_ Feynman first #############################
ALGORITHMS = ["AIFeynman", "Operon", "AFP_FE", "AFP", "BSR", "DSR", "EPLEX", "FEAT", "FFX", "GP-GOMEA", "ITEA", "MRGP", "SBP-GP", "gplearn"   ]
data_groups = ["Feynman", "Strogatz"]
noise_levels = [0.0, 0.001, 0.01 ]

## index of groups
data_groups_index = 1 # data_groups[ data_groups_index ]
noise_levels_index = 2 # noise_levels[ noise_levels_index ]
criterion_column = "symbolic_solution_rate_(%)" #"symbolic_solution_rate_(%)" #accuracy_solution
criterion_feynman_column = "correct"#  R2>0.999 correct
################# dataset ####################################

results_groundtruth = pd.read_csv("groundtruth.csv", sep="\t")
 #conversion:
print(results_groundtruth.info())
# 
results_groundtruth_noise = results_groundtruth[(results_groundtruth["target_noise"] == noise_levels[ noise_levels_index ]) & (results_groundtruth["data_group"] == data_groups[ data_groups_index ])]
results_groundtruth_noise = results_groundtruth_noise.drop(["label", "label2"], axis = 1)
########################## aggregate ##################################################################################################
results_groundtruth_noise_groupby_alg = results_groundtruth_noise.groupby(["algorithm"]).aggregate({ criterion_column : "mean" }) # symbolic_solution_rate_(%)
results_groundtruth_noise_groupby_alg_ri = results_groundtruth_noise_groupby_alg.reset_index()
#results_groundtruth_noise_groupby_alg_ri

results_groundtruth_noise_required = results_groundtruth_noise[["algorithm", "dataset", criterion_column]]
#results_groundtruth_noise_required
########## feynman or stogatz results ==> include ##########################

def feynman_or_strogatz(filename, noise_level):
    
    file_name = ""
    if filename == "Feynman":
        file_name = "feynman_results.csv"
    else:
        file_name = "strogatz_results.csv"
    
    feynman_results = pd.read_csv(file_name, delimiter=",", encoding='utf-16') #, encoding='cp1252')
    #print(feynman_results)
    feynman_results_groupby = feynman_results.groupby(["instance"]).aggregate({criterion_feynman_column: "sum", criterion_feynman_column +".1" : "sum", criterion_feynman_column + ".2" : "sum" })
    feynman_results_groupby_ri = feynman_results_groupby.reset_index()
    feynman_results_groupby_ri['algorithm'] = feynman_results_groupby_ri.apply(lambda x: 'Rils-Rols', axis = 1)

    if noise_level == 0.0: 
        feynman_results_groupby_ri = feynman_results_groupby_ri[["algorithm", "instance", criterion_feynman_column]]
    elif noise_level == 0.001:
        feynman_results_groupby_ri = feynman_results_groupby_ri[["algorithm", "instance", criterion_feynman_column + ".1"]]
    else:
        feynman_results_groupby_ri = feynman_results_groupby_ri[["algorithm", "instance", criterion_feynman_column + ".2"]]
    
    feynman_results_groupby_ri.columns = ["algorithm", "dataset",  criterion_feynman_column]
    
    if criterion_column != "accuracy_solution":
        feynman_results_groupby_ri[criterion_feynman_column] = feynman_results_groupby_ri[ criterion_feynman_column ].apply(lambda x: x * 10 )
    else:
        feynman_results_groupby_ri[criterion_feynman_column] = feynman_results_groupby_ri[ criterion_feynman_column ].apply(lambda x: x / 10 )

    return feynman_results_groupby_ri

#results_groundtruth_noise_required

####################################### Strogatz and Feynman dataset include for Rils-Rols #################################################
feynman_results_noise = pd.DataFrame()

if data_groups[ data_groups_index ] == "Feynman": 
    feynman_results_noise = feynman_or_strogatz(data_groups[0], noise_levels[ noise_levels_index ])
else:
    feynman_results_noise = feynman_or_strogatz(data_groups[1], noise_levels[ noise_levels_index ])

##################### merge datasets columns: algorithms ==> rows dataset: symbolc_sol. rate ##########################################################

print(results_groundtruth_noise_required )

########## results_groundtruth_noise_required: split by algorithm ###################################
results_groundtruth_by_algorithm = {}

for algorithm in ALGORITHMS:
    results_groundtruth_noise_alg = results_groundtruth_noise_required[results_groundtruth_noise_required["algorithm"] == algorithm]
    results_groundtruth_by_algorithm[algorithm] = results_groundtruth_noise_alg
    results_groundtruth_by_algorithm[algorithm].drop(columns = ["algorithm"], axis = 1, inplace=True)
    results_groundtruth_by_algorithm[algorithm].columns = ["dataset", algorithm ]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7270 entries, 0 to 7269
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   label                       7270 non-null   object 
 1   label2                      7270 non-null   object 
 2   algorithm                   7270 non-null   object 
 3   dataset                     7270 non-null   object 
 4   target_noise                7270 non-null   float64
 5   data_group                  7270 non-null   object 
 6   symbolic_solution_rate_(%)  7270 non-null   float64
 7   accuracy_solution           7270 non-null   float64
 8   rmse_test                   7270 non-null   float64
 9   log_mse_test                7270 non-null   float64
dtypes: float64(5), object(5)
memory usage: 568.1+ KB
None
     algorithm              dataset  symbolic_solution_rate_(%)
466        AFP     strogatz_bacres1                         0.0
470        AFP     str

/tmp/ipykernel_18344/3575737829.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_groundtruth_by_algorithm[algorithm].drop(columns = ["algorithm"], axis = 1, inplace=True)
/tmp/ipykernel_18344/3575737829.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_groundtruth_by_algorithm[algorithm].drop(columns = ["algorithm"], axis = 1, inplace=True)
/tmp/ipykernel_18344/3575737829.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [173]:
########## transform into 10 runs:
results_groundtruth_by_algorithms_expanded = {}
results_datasets =  results_groundtruth_by_algorithm["AFP"]['dataset'].to_numpy() 

for algorithm in ALGORITHMS:# iterrate throug the algorithms
    
    results_groundtruth_by_algorithm_expanded = pd.DataFrame()
    for index, row in results_groundtruth_by_algorithm[ algorithm ].iterrows():
    
        number_found = 0
    
        if row[ algorithm ] - int(row[ algorithm ]) == 0.0: # natural number
            number_found = row[ algorithm ] // 10
        else:
            number_found = int(row[ algorithm ] / 12.5) 
            
        for i in range(10):
            row_add = {}
        
            if i < number_found:
                row_add = { "index" : i, "dataset" : row['dataset'], algorithm : 1} 
            else:
                row_add = { "index" : i, "dataset" : row['dataset'], algorithm : 0} 
            ## add 
            results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
    
    #### not finised runs: add 0s everywhere 
    results_expanded_datasets =  results_groundtruth_by_algorithm_expanded['dataset'].to_numpy()
    print(results_expanded_datasets)
    
    for dataset in results_datasets:
        if dataset not in results_expanded_datasets:
            for i in range(10):
                row_add = {}
                row_add = { "index" : i, "dataset" : row['dataset'], algorithm : 0} 
                results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
    
    results_groundtruth_by_algorithms_expanded[algorithm] = results_groundtruth_by_algorithm_expanded
    
    

/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed 

/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed 

/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed 

/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed 

/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed 

...

/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed 

/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed 

/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed 

['strogatz_bacres1' 'strogatz_bacres1' 'strogatz_bacres1'
 'strogatz_bacres1' 'strogatz_bacres1' 'strogatz_bacres1'
 'strogatz_bacres1' 'strogatz_bacres1' 'strogatz_bacres1'
 'strogatz_bacres1' 'strogatz_bacres2' 'strogatz_bacres2'
 'strogatz_bacres2' 'strogatz_bacres2' 'strogatz_bacres2'
 'strogatz_bacres2' 'strogatz_bacres2' 'strogatz_bacres2'
 'strogatz_bacres2' 'strogatz_bacres2' 'strogatz_barmag1'
 'strogatz_barmag1' 'strogatz_barmag1' 'strogatz_barmag1'
 'strogatz_barmag1' 'strogatz_barmag1' 'strogatz_barmag1'
 'strogatz_barmag1' 'strogatz_barmag1' 'strogatz_barmag1'
 'strogatz_barmag2' 'strogatz_barmag2' 'strogatz_barmag2'
 'strogatz_barmag2' 'strogatz_barmag2' 'strogatz_barmag2'
 'strogatz_barmag2' 'strogatz_barmag2' 'strogatz_barmag2'
 'strogatz_barmag2' 'strogatz_glider1' 'strogatz_glider1'
 'strogatz_glider1' 'strogatz_glider1' 'strogatz_glider1'
 'strogatz_glider1' 'strogatz_glider1' 'strogatz_glider1'
 'strogatz_glider1' 'strogatz_glider1' 'strogatz_glider2'
 'strogatz_gli

/tmp/ipykernel_18344/2624140992.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_groundtruth_by_algorithm_expanded = results_groundtruth_by_algorithm_expanded.append(row_add,  ignore_index=True)


In [174]:
#results_groundtruth_by_algorithms_expanded["AFP"][results_groundtruth_by_algorithms_expanded["AFP"]["dataset"] == "feynman_III_12_43"] 
#results_datasets =  results_groundtruth_by_algorithms_expanded["AFP"]['dataset'].to_numpy() 
#print('feynman_III_10_11' in results_datasets)
#results_groundtruth_by_algorithm["AFP"]

In [175]:
########## Feynamn/Strogatz: reduction
print(feynman_results_noise)
data_all_results = pd.DataFrame()
feynman_results_noise.drop(columns = ["algorithm"], axis = 1, inplace=True)
feynman_results_noise.columns = ["dataset", "Rils-Rols"]


    algorithm              dataset  correct
0   Rils-Rols     strogatz_bacres1        0
1   Rils-Rols     strogatz_bacres2        0
2   Rils-Rols     strogatz_barmag1        0
3   Rils-Rols     strogatz_barmag2       30
4   Rils-Rols     strogatz_glider1       70
5   Rils-Rols     strogatz_glider2       20
6   Rils-Rols         strogatz_lv1       20
7   Rils-Rols         strogatz_lv2       10
8   Rils-Rols   strogatz_predprey1        0
9   Rils-Rols   strogatz_predprey2        0
10  Rils-Rols  strogatz_shearflow1        0
11  Rils-Rols  strogatz_shearflow2       60
12  Rils-Rols        strogatz_vdp1        0
13  Rils-Rols        strogatz_vdp2      100


In [176]:
############# expansion:

feynman_results_noise_expanded = pd.DataFrame()
for index, row in feynman_results_noise.iterrows():

    number_found = row[ "Rils-Rols" ] // 10
    for i in range(10):
        row_add = {}
        
        if i < number_found:
            row_add = { "index" : i, "dataset" : row['dataset'], "Rils-Rols": 1} 
        else:
            row_add = { "index" : i, "dataset" : row['dataset'], "Rils-Rols": 0} 
        ## add 
        feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)

### expanded        
feynman_results_noise_expanded        
        

/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_

/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_

/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_

/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_results_noise_expanded = feynman_results_noise_expanded.append(row_add,  ignore_index=True)
/tmp/ipykernel_18344/651158986.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feynman_

,index,dataset,Rils-Rols
0,0,strogatz_bacres1,0
1,1,strogatz_bacres1,0
2,2,strogatz_bacres1,0
3,3,strogatz_bacres1,0
4,4,strogatz_bacres1,0
...,...,...,...
135,5,strogatz_vdp2,1
136,6,strogatz_vdp2,1
137,7,strogatz_vdp2,1
138,8,strogatz_vdp2,1


In [177]:
### MERGE:
data_all_results = feynman_results_noise.copy()
### merge results
for algorithm in ALGORITHMS:
    data_all_results = data_all_results.merge(results_groundtruth_by_algorithm[algorithm], on = ["dataset"])   
#### save to .csv
data_all_results['noise'] = data_all_results.apply(lambda x: noise_levels[ noise_levels_index ], axis = 1) 
data_all_results.to_csv( data_groups[ data_groups_index ]  + "_noise_" +  str(noise_levels[ noise_levels_index ] ).replace( ".", "_" ) + criterion_column + ".csv", sep=",")

print(data_groups[ data_groups_index ]  + "_noise_" +  str(noise_levels[ noise_levels_index ] ).replace( ".", "_" ) + ".csv")
 

Strogatz_noise_0_01.csv


In [178]:
########## merge expanded:
data_all_results_expanded = feynman_results_noise_expanded.copy()

for algorithm in ALGORITHMS:
    data_all_results_expanded = data_all_results_expanded.merge(results_groundtruth_by_algorithms_expanded[algorithm], on = ["dataset", "index"])   
#### save to .csv
data_all_results_expanded ###### expansions
data_all_results_expanded.to_csv( data_groups[ data_groups_index ]  + "_noise_" +  str(noise_levels[ noise_levels_index ] ).replace( ".", "_" ) + criterion_column + "-expanded.csv", sep=",")


In [179]:
#### statistics 
from scipy import stats
print(data_all_results['Rils-Rols'].mean())
print(data_all_results['Operon'].mean())

###  the Wilcoxon signed rank test
#here was a statistically significant difference if  p < 0.05; otherwise not 
stats.wilcoxon(data_all_results['Rils-Rols'], data_all_results['AFP_FE'])## reject hypothesis 



22.142857142857142
0.7142857142857143


/home/marko/.local/lib/python3.8/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/home/marko/.local/lib/python3.8/site-packages/scipy/stats/_morestats.py:3159: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


WilcoxonResult(statistic=5.5, pvalue=0.1471240986757617)

In [180]:
#f_s_data = pd.read_csv("feynman_strogatz_noise0_001.csv")
#
#stats.ranksums(f_s_data["Rils_Rols"], f_s_data["DSR"])

In [181]:
#f_s_data.head(50)